## Decision Trees

### A decision tree has a series of nodes. 

### To make a prediction you start at the trees root node. If the node has children you decend to the child depending on how your input compares with this nodes split point. If the node does not have children for a regression tree you guess the mean, for a tree classifier you guess the most common class.

### The split point of a node in a decision is chosen such that information gain is maximized.

Intuitively information gain can be see as how much we lower the combined spread of a distribution by splitting it at a specific location.

$$H(X) = -\sum p(X)\log p(X)\\ Information\ Gain\; I(X,Y)= H(X)-H(X|Y)$$

In [4]:
import numpy as np

In [94]:
class DecisionTreeClassifier():
    def __init__(self,X:np.ndarray,y:np.ndarray,max_depth,num_splits):
        '''
        Decision Tree Classifier:
        Parameters:
            X: An array of numerical factors
            y: The responding variable, must be categorical
            max_depth: The maximum number of nodes from root to leaves in each estimator
            num_splits: The number of splits to be tested for each factor at each node
        '''
        self.tree = self.Node_(X,y,self,num_splits,max_depth)
    def predict(self,X):
        '''
        Predict:
        Predicts using the established tree
        
        Parameters:
            X: An array of numerical factors
            
        Returns:
            An array of predictions.
        '''
        total_predictions=[]
        for x in X:
            node = self.tree
            while node.prediction_value is None:
                #decend tree
                if x[node.split_characteristic] <= node.split: node = node.left
                else: node = node.right
            total_predictions.append(node.prediction_value)
        return total_predictions
    class Node_():
        def __init__(self, X,y,tree, num_splits, max_depth,current_depth=0):
            self.left = None
            self.right = None

            self.tree = tree
            #calculate entropy
            classes = np.unique(y)
            if len(classes) == 1:
                self.prediction_value = classes[0]
                return #early stop
            p_classes = []
            for class_ in classes:
                p_class = np.sum(y==class_)/len(y)
                p_classes.append(p_class)

            self.entropy = -np.sum(np.array([p_class*np.log2(p_class) for p_class in p_classes]))


            info_gains = [] # used to find best split
            total_splits = []

            for column_index in range(X.shape[1]):
                curr_column = X[:,column_index]

                #pick 10 random potential split points

                random_splits = np.random.random_sample(num_splits,)*(curr_column.max()-curr_column.min())\
                                + curr_column.min()
                total_splits = np.concatenate([total_splits, random_splits],axis=0)

                # decide on best split using information gain
                for split in random_splits:

                    y_lower = y[curr_column<=split]
                    y_higher = y[curr_column>split]
                    lower_p_classes = []
                    higher_p_classes = []
                    #find entropy of each split
                    for class_ in classes:
                        lower_p_class = np.sum(y_lower==class_)/len(y_lower)
                        lower_p_classes.append(lower_p_class)    
                        higher_p_class = np.sum(y_higher==class_)/len(y_higher)
                        higher_p_classes.append(higher_p_class)    
                    lower_entropy = -np.sum(np.array([p_class*np.log2(p_class) for p_class in lower_p_classes]))
                    higher_entropy = -np.sum(np.array([p_class*np.log2(p_class) for p_class in higher_p_classes]))

                    info_gains.append(self.entropy -  higher_entropy - lower_entropy)
            # split using best splitpoint
            arg_max = np.argmax(np.array(info_gains))
            self.split_characteristic = arg_max // len(random_splits)
            final_split = total_splits[arg_max]
            self.split = final_split
            # also split X and y
            final_X_lower = X[X[:,self.split_characteristic]<=self.split, :]
            final_X_higher = X[X[:,self.split_characteristic]>self.split, :]
            final_y_lower = y[X[:,self.split_characteristic]<=self.split]
            final_y_higher = y[X[:,self.split_characteristic]>self.split]
            #assign children
            if current_depth<max_depth:
                self.left = self.tree.Node_(X=final_X_lower,\
                                            y=final_y_lower,\
                                            tree=self.tree,\
                                            num_splits=num_splits,\
                                            max_depth=max_depth,\
                                            current_depth=current_depth+1)
                self.right = self.tree.Node_(X=final_X_higher,\
                                            y=final_y_higher,\
                                            tree=self.tree,\
                                            num_splits=num_splits,\
                                            max_depth=max_depth,\
                                            current_depth=current_depth+1)
                self.prediction_value = None
            #asign property to be predicted
            else:
                self.prediction_value = classes[np.argmax(np.array(p_classes))]

In [218]:
class DecisionTreeRegressor():
        def __init__(self,X:np.ndarray,y:np.ndarray,max_depth,num_splits):
            '''
            Decision Tree Classifier:
            Parameters:
                X: An array of numerical factors
                y: The responding variable, must be continuous
                max_depth: The maximum number of nodes from root to leaves in each estimator
                num_splits: The number of splits to be tested for each factor at each node
            '''
            self.tree = self.Node_(X,y,max_depth,num_splits,self,0)

        def predict(self,X):
            '''
            Predict:
            Predicts using the established tree

            Parameters:
                X: An array of numerical factors

            Returns:
                An array of predictions.
            '''            
            total_predictions=[]
            for x in X:
                node = self.tree
                while node.prediction_value is None:
                    #decend tree
                    if x[node.split_characteristic] <= node.split: node = node.left
                    else: node = node.right
                total_predictions.append(node.prediction_value)
            return total_predictions
        class Node_():
            def __init__(self, X,y, max_depth, num_splits, tree=None,current_depth=0):
                self.left = None
                self.right = None
                self.tree = tree
                #calculate entropy
                classes = np.unique(y)
                if len(classes) == 1:
                    self.prediction_value = classes[0]
                    return #early stopping if we have prematurely gotten a 'Pure Node'
                # get the proportions of each class in y
                p_classes = []
                for class_ in classes:
                    p_class = np.sum(y==class_)/len(y)
                    p_classes.append(p_class)

                variances = [] # used to find best split
                total_splits = []
                for column_index in range(X.shape[1]):
                    curr_column = X[:,column_index]

                    #pick 10 random potential split points
                    random_splits = np.random.random_sample(num_splits,)*(curr_column.max()-curr_column.min())\
                                    + curr_column.min()
                    total_splits = np.concatenate([total_splits, random_splits],axis=0)
    
                    # decide on best split using information gain
                    for split in random_splits:
                        #find combined mse for each split

                        y_lower = y[curr_column<=split]
                        y_higher = y[curr_column>split]
                        y_lower_mean = y_lower.mean()
                        y_higher_mean = y_higher.mean()
         
        
                        variances.append(((y_lower-y_lower_mean)**2).sum()+((y_higher-y_higher_mean)**2).sum())

                # split using best splitpoint
                arg_min = np.argmin(np.array(variances))

                self.split_characteristic = arg_min // len(random_splits)

                final_split = total_splits[arg_min]

                self.split = final_split

                final_X_lower = X[X[:,self.split_characteristic]<=self.split, :]
                final_X_higher = X[X[:,self.split_characteristic]>self.split, :]
                final_y_lower = y[X[:,self.split_characteristic]<=self.split]
                final_y_higher = y[X[:,self.split_characteristic]>self.split]
                #assign children
                if current_depth<max_depth:
                    self.left = self.tree.Node_(final_X_lower,\
                                                final_y_lower,\
                                                max_depth,\
                                                num_splits,\
                                                self.tree,\
                                                current_depth+1)
                    self.right = self.tree.Node_(final_X_higher,\
                                                 final_y_higher,\
                                                 max_depth,\
                                                 num_splits,\
                                                 self.tree,\
                                                 current_depth+1)
                    self.prediction_value = None
                else:
                    #value to predict with
                    self.prediction_value = y.mean()

In [158]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
dataset = load_breast_cancer()

In [159]:
data = dataset['data']
target = dataset['target']

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [160]:
tree =DecisionTreeClassifier(data_train,target_train,max_depth=4,num_splits=10)

/Users/alan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in log2
/Users/alan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
/Users/alan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in log2
/Users/alan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in double_scalars


In [161]:
predictions = tree.predict(data_test)

In [162]:
#print recall as this is critical for medical tests

TP = np.array([prediction and target for prediction,target in zip(predictions,target_test)])
FN = np.array([ target and not prediction  for prediction,target in zip(predictions,target_test)])
print(TP.sum()/(FN.sum()+TP.sum()))


1.0


In [163]:
predictions[:20]

[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

In [164]:
target_test[:20]

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0])

In [165]:
FN

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

Recall is good, how is our precision?

In [166]:
TP = np.array([prediction and target for prediction,target in zip(predictions,target_test)])
FP = np.array([ prediction and not target  for prediction,target in zip(predictions,target_test)])
print(TP.sum()/(FP.sum()+TP.sum()))

0.6893203883495146


Not the best, but we can work on this.

Lets try some regression

In [167]:
from sklearn.datasets import load_boston
dataset = load_boston()

In [168]:
data = dataset['data']
target = dataset['target']

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [210]:
regression_tree =DecisionTreeRegressor(data_train,target_train,max_depth=7,num_splits=10)

/Users/alan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: Mean of empty slice.
/Users/alan/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [211]:
predictions = np.array(regression_tree.predict(data_test))

In [212]:
predictions[:10]

array([24.3       , 31.85555556, 15.6       , 21.10555556, 15.18      ,
       20.21176471, 19.48181818, 15.6       , 22.13333333, 19.26285714])

In [213]:
target_test[:10]

array([23.6, 32.4, 13.6, 22.8, 16.1, 20. , 17.8, 14. , 19.6, 16.8])

In [214]:
from sklearn.metrics import mean_squared_error

In [215]:
mean_squared_error(predictions,target_test)

14.18004748583447

In [216]:
max(predictions-target_test)

26.3

In [217]:
min(predictions-target_test)

-7.199999999999999

Looks like we have some bad errors but our predictions are sensible.